In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
@author: M Arshad Zahangir Chowdhury

Tune hyperparameters of VOC_net model.

"""

%matplotlib inline 

import sys
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from ipywidgets import interactive
import seaborn as sns  #heat map
import glob # batch processing of images

if '../../' not in sys.path:
    sys.path.append('../../')

import math
from scipy import signal
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve 
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score

import itertools

from aimos.misc.utils import classifier_internals
from aimos.misc.utils import clf_post_processor


from aimos.spectral_datasets.IR_datasets import IR_data
from aimos.spectral_datasets.IR_datasets import spectra_to_img
from aimos.spectral_datasets.THz_datasets import THz_data

from aimos.misc.aperture import publication_fig
from aimos.misc.voc_net_utils import multiclass_roc_auc_score
from aimos.misc.voc_net_utils import plot_raw_scores
from aimos.misc.voc_net_utils import simple_spectrum_fig
from aimos.misc.voc_net_utils import simple_plot_raw_scores

from aimos.misc.voc_net_utils import plot_sequential_group_prediction



import tensorflow as tf
from tensorflow.keras import datasets, layers, models

GPU_mem_limit=1.0
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=GPU_mem_limit*1000.0)])

    except RuntimeError as e:
        print(e)        



# !pip install git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
import tensorflow_docs.modeling
from tensorflow.keras import regularizers

from aimos.models.voc_net_models import get_callbacks
from aimos.models.voc_net_models import get_optimizer
from aimos.models.voc_net_models import compile_and_fit


from aimos.models.voc_net_models import C1f1k3_AP1_D12
from aimos.models.voc_net_models import C1f1k3_MP1_D12

from aimos.models.voc_net_models import C2f1k3_AP1_D12
from aimos.models.voc_net_models import C2f1k3_AP1_D48_D12
from aimos.models.voc_net_models import C2f1k3_AP2_D48_D12

from aimos.models.voc_net_models import C2f3k3_AP1_D48_D12
from aimos.models.voc_net_models import C2f3k3_AP1_D6_D12

from aimos.models.voc_net_models import C1f1k3_AP1_RD50_D12
from aimos.models.voc_net_models import C1f1k3_AP1_D48_RL1_D12
from aimos.models.voc_net_models import C2f3k3_AP1_D48_RD50_D12
from aimos.models.voc_net_models import C2f3k3_AP1_D48_RL1_D12
from aimos.models.voc_net_models import C2f3k3_AP1_D48_RL1_RD50_D12

    

from tensorflow import keras
import keras_tuner as kt

#Set random seed
# os.environ['PYTHONHASHSEED'] = str(42)
tf.random.set_seed(42)  
# np.random.seed(42)

2022-07-07 12:59:49.632684: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-07-07 12:59:50.496051: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-07-07 12:59:50.526016: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: Quadro RTX 4000 computeCapability: 7.5
coreClock: 1.545GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 387.49GiB/s
2022-07-07 12:59:50.526078: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-07-07 12:59:50.532176: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-07-07 12:59:50.532277: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so

In [2]:
s = THz_data(resolution=0.016, verbosity = False)
s.load_THz_data()
# s.dataset_info()
X = s.spectra
y = s.targets

X=np.expand_dims(X,-1)

In [3]:
#split intro train and test set

#seeds used 123,237, 786
from sklearn.model_selection import train_test_split

TRAIN_SIZE=0.70
TEST_SIZE=1-TRAIN_SIZE

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=TRAIN_SIZE,
                                                   test_size=TEST_SIZE,
                                                   random_state=786,
                                                   stratify=y
                                                   )

print("All:", np.bincount(y) / float(len(y))*100  )
print("Training:", np.bincount(y_train) / float(len(y_train))*100  )
print("Testing:", np.bincount(y_test) / float(len(y_test))*100  )


All: [8.33333333 8.33333333 8.33333333 8.33333333 8.33333333 8.33333333
 8.33333333 8.33333333 8.33333333 8.33333333 8.33333333 8.33333333]
Training: [8.35148874 8.35148874 8.35148874 8.35148874 8.35148874 8.35148874
 8.35148874 8.2788671  8.35148874 8.35148874 8.2788671  8.2788671 ]
Testing: [8.29103215 8.29103215 8.29103215 8.29103215 8.29103215 8.29103215
 8.29103215 8.46023689 8.29103215 8.29103215 8.46023689 8.46023689]


In [4]:
def thz_model(hp):

    model = models.Sequential()
    
    hp_filters= hp.Int('filters', min_value=2, max_value=4, step=1)
#     hp_kernel_size= hp.Int('kernel_size', min_value=3, max_value=5, step=1)
#     hp_pool_size=hp.Int('pool_size', min_value=2, max_value=3, step=1)
#     hp_stride_size=hp.Int('stride_size', min_value=2, max_value=3, step=1)
    hp_units= hp.Int('units', min_value=24, max_value=48, step=6)
#     hp_dropout = hp.Choice('dropout', values=[0.3, 0.5, 0.7])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # C1 Convolutional Layer
    model.add(layers.Conv1D(filters = hp_filters , kernel_size=3, activation='relu', input_shape=(229, 1), name = 'C1') )

    # S2 Subsampling Layer
    model.add(layers.AveragePooling1D(pool_size = 2, strides = 2, padding = 'valid', name = 'S2'))
    
    # C3 Convolutional Layer
    model.add(layers.Conv1D(filters = hp_filters , kernel_size=3, activation='relu', name = 'C3') )

    # Flatten the CNN output to feed it with fully connected layers
    model.add(layers.Flatten())
    
    model.add(layers.Dense(hp_units, activation='relu')) 
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(12))  # number of dense layer would be equal to number of classess
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=[
              tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True, name='SparseCatCrossentropy'),
              'accuracy'])
    
    model.summary()
    
    return model

In [5]:
tuner = kt.Hyperband(thz_model,
                     objective='accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='keras_hp_tuner',
                     project_name='thz-cnn')

INFO:tensorflow:Reloading Oracle from existing project keras_hp_tuner/thz-cnn/oracle.json
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
C1 (Conv1D)                  (None, 227, 2)            8         
_________________________________________________________________
S2 (AveragePooling1D)        (None, 113, 2)            0         
_________________________________________________________________
C3 (Conv1D)                  (None, 111, 2)            14        
_________________________________________________________________
flatten (Flatten)            (None, 222)               0         
_________________________________________________________________
dense (Dense)                (None, 24)                5352      
_________________________________________________________________
dropout (Dropout)            (None, 24)                0         
________________________________

2022-07-07 12:59:52.679286: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-07 12:59:52.680229: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: Quadro RTX 4000 computeCapability: 7.5
coreClock: 1.545GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 387.49GiB/s
2022-07-07 12:59:52.681113: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2022-07-07 12:59:52.681194: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-07-07 12:59:53.086049: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect Str

In [6]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='SparseCatCrossentropy', patience=5)

In [7]:
# tuner.search(X, y, epochs=200, validation_split=0.3, callbacks=[stop_early])
tuner.search(X, y, epochs=200, validation_data=(x_test, y_test), callbacks=[stop_early])


# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=2)[0]



INFO:tensorflow:Oracle triggered exit


In [8]:
print("""The hyperparameter search is complete.""")
# print(f"""
# hp_filters_1= {best_hps.get('filters')}
# hp_kernel_size= {best_hps.get('kernel_size')}
# hp_pool_size= {best_hps.get('pool_size')}
# hp_stride_size={best_hps.get('stride_size')}
# hp_units= {best_hps.get('units')}
# hp_dropout = {best_hps.get('dropout')}
# hp_learning_rate =  {best_hps.get('learning_rate')}
# """)

print(f"""
hp_filters= {best_hps.get('filters')}


hp_units= {best_hps.get('units')}

hp_learning_rate =  {best_hps.get('learning_rate')}
""")



The hyperparameter search is complete.

hp_filters= 3


hp_units= 48

hp_learning_rate =  0.01



In [9]:
best_hps.space

[Int(name: "filters", min_value: 2, max_value: 4, step: 1, sampling: None, default: 2),
 Int(name: "units", min_value: 24, max_value: 48, step: 6, sampling: None, default: 24),
 Choice(name: "learning_rate", values: [0.01, 0.001, 0.0001], ordered: True, default: 0.01)]

In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=200, validation_data=(x_test, y_test))

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
C1 (Conv1D)                  (None, 227, 3)            12        
_________________________________________________________________
S2 (AveragePooling1D)        (None, 113, 3)            0         
_________________________________________________________________
C3 (Conv1D)                  (None, 111, 3)            30        
_________________________________________________________________
flatten_1 (Flatten)          (None, 333)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 48)                16032     
_________________________________________________________________
dropout_1 (Dropout)          (None, 48)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 12)               

2022-07-07 13:00:03.476771: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-07 13:00:03.497642: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3600000000 Hz
2022-07-07 13:00:03.988891: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-07-07 13:00:04.233095: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8204
2022-07-07 13:00:04.689231: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


 1/44 [..............................] - ETA: 1:18 - loss: 2.4642 - SparseCatCrossentropy: 2.4642 - accuracy: 0.0312

2022-07-07 13:00:05.156000: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


44/44 [==============================] - 2s 12ms/step - loss: 1.0290 - SparseCatCrossentropy: 1.0064 - accuracy: 0.7248 - val_loss: 0.1309 - val_SparseCatCrossentropy: 0.1310 - val_accuracy: 0.9831
Epoch 2/200
44/44 [==============================] - 0s 7ms/step - loss: 0.1921 - SparseCatCrossentropy: 0.1880 - accuracy: 0.9506 - val_loss: 0.0470 - val_SparseCatCrossentropy: 0.0473 - val_accuracy: 0.9949
Epoch 3/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0980 - SparseCatCrossentropy: 0.0995 - accuracy: 0.9826 - val_loss: 0.0304 - val_SparseCatCrossentropy: 0.0304 - val_accuracy: 0.9966
Epoch 4/200
44/44 [==============================] - 0s 7ms/step - loss: 0.1575 - SparseCatCrossentropy: 0.1541 - accuracy: 0.9731 - val_loss: 0.0179 - val_SparseCatCrossentropy: 0.0183 - val_accuracy: 1.0000
Epoch 5/200
44/44 [==============================] - 0s 7ms/step - loss: 0.1309 - SparseCatCrossentropy: 0.1280 - accuracy: 0.9739 - val_loss: 0.0190 - val_SparseCatCrossentro

44/44 [==============================] - 0s 7ms/step - loss: 0.0203 - SparseCatCrossentropy: 0.0198 - accuracy: 0.9964 - val_loss: 0.0022 - val_SparseCatCrossentropy: 0.0021 - val_accuracy: 1.0000
Epoch 41/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0177 - SparseCatCrossentropy: 0.0173 - accuracy: 0.9942 - val_loss: 7.7580e-04 - val_SparseCatCrossentropy: 7.5815e-04 - val_accuracy: 1.0000
Epoch 42/200
44/44 [==============================] - 0s 8ms/step - loss: 0.0153 - SparseCatCrossentropy: 0.0150 - accuracy: 0.9971 - val_loss: 5.8284e-04 - val_SparseCatCrossentropy: 5.6970e-04 - val_accuracy: 1.0000
Epoch 43/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0175 - SparseCatCrossentropy: 0.0171 - accuracy: 0.9927 - val_loss: 9.4120e-04 - val_SparseCatCrossentropy: 9.2694e-04 - val_accuracy: 1.0000
Epoch 44/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0190 - SparseCatCrossentropy: 0.0185 - accuracy: 0.9949 - val_loss: 8.019

Epoch 78/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0223 - SparseCatCrossentropy: 0.0218 - accuracy: 0.9949 - val_loss: 3.4710e-05 - val_SparseCatCrossentropy: 3.3842e-05 - val_accuracy: 1.0000
Epoch 79/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0236 - SparseCatCrossentropy: 0.0231 - accuracy: 0.9942 - val_loss: 8.1616e-05 - val_SparseCatCrossentropy: 7.9573e-05 - val_accuracy: 1.0000
Epoch 80/200
44/44 [==============================] - 0s 8ms/step - loss: 0.0116 - SparseCatCrossentropy: 0.0113 - accuracy: 0.9971 - val_loss: 8.3661e-05 - val_SparseCatCrossentropy: 8.1448e-05 - val_accuracy: 1.0000
Epoch 81/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0046 - SparseCatCrossentropy: 0.0045 - accuracy: 0.9978 - val_loss: 5.4689e-05 - val_SparseCatCrossentropy: 5.3328e-05 - val_accuracy: 1.0000
Epoch 82/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0087 - SparseCatCrossentropy: 0.0195 - accuracy: 0.9

44/44 [==============================] - 0s 7ms/step - loss: 0.0435 - SparseCatCrossentropy: 0.0425 - accuracy: 0.9956 - val_loss: 2.1186e-04 - val_SparseCatCrossentropy: 2.0601e-04 - val_accuracy: 1.0000
Epoch 116/200
44/44 [==============================] - 0s 6ms/step - loss: 0.0205 - SparseCatCrossentropy: 0.0200 - accuracy: 0.9935 - val_loss: 5.2909e-04 - val_SparseCatCrossentropy: 5.1438e-04 - val_accuracy: 1.0000
Epoch 117/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0119 - SparseCatCrossentropy: 0.0116 - accuracy: 0.9956 - val_loss: 8.1268e-05 - val_SparseCatCrossentropy: 7.9040e-05 - val_accuracy: 1.0000
Epoch 118/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0016 - SparseCatCrossentropy: 0.0015 - accuracy: 1.0000 - val_loss: 1.9915e-04 - val_SparseCatCrossentropy: 1.9360e-04 - val_accuracy: 1.0000
Epoch 119/200
44/44 [==============================] - 0s 6ms/step - loss: 0.0111 - SparseCatCrossentropy: 0.0109 - accuracy: 0.9971 - val

44/44 [==============================] - 0s 7ms/step - loss: 0.0276 - SparseCatCrossentropy: 0.0270 - accuracy: 0.9949 - val_loss: 2.8633e-05 - val_SparseCatCrossentropy: 2.7839e-05 - val_accuracy: 1.0000
Epoch 153/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0084 - SparseCatCrossentropy: 0.0082 - accuracy: 0.9971 - val_loss: 4.4414e-05 - val_SparseCatCrossentropy: 4.3175e-05 - val_accuracy: 1.0000
Epoch 154/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0206 - SparseCatCrossentropy: 0.0202 - accuracy: 0.9942 - val_loss: 1.7203e-04 - val_SparseCatCrossentropy: 1.6722e-04 - val_accuracy: 1.0000
Epoch 155/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0087 - SparseCatCrossentropy: 0.0085 - accuracy: 0.9985 - val_loss: 4.7008e-05 - val_SparseCatCrossentropy: 4.5695e-05 - val_accuracy: 1.0000
Epoch 156/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0077 - SparseCatCrossentropy: 0.0075 - accuracy: 0.9956 - val

Epoch 190/200
44/44 [==============================] - 0s 6ms/step - loss: 0.0112 - SparseCatCrossentropy: 0.0110 - accuracy: 0.9978 - val_loss: 1.0344e-05 - val_SparseCatCrossentropy: 1.0055e-05 - val_accuracy: 1.0000
Epoch 191/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0474 - SparseCatCrossentropy: 0.0463 - accuracy: 0.9906 - val_loss: 1.8850e-04 - val_SparseCatCrossentropy: 1.8323e-04 - val_accuracy: 1.0000
Epoch 192/200
44/44 [==============================] - 0s 6ms/step - loss: 0.0405 - SparseCatCrossentropy: 0.0396 - accuracy: 0.9891 - val_loss: 3.6662e-05 - val_SparseCatCrossentropy: 3.5637e-05 - val_accuracy: 1.0000
Epoch 193/200
44/44 [==============================] - 0s 6ms/step - loss: 0.0073 - SparseCatCrossentropy: 0.0071 - accuracy: 0.9971 - val_loss: 4.6879e-06 - val_SparseCatCrossentropy: 4.5568e-06 - val_accuracy: 1.0000
Epoch 194/200
44/44 [==============================] - 0s 7ms/step - loss: 0.0122 - SparseCatCrossentropy: 0.0119 - accuracy

# notebook ends

In [ ]:
def voc_net():

    model = models.Sequential()

    # C1 Convolutional Layer
    model.add(layers.Conv1D(filters = 3 , kernel_size=3, activation='relu', input_shape=(229, 1), name = 'C1') )

    # S2 Subsampling Layer
    model.add(layers.AveragePooling1D(pool_size = 2, strides = 2, padding = 'valid', name = 'S2'))
    
    # C3 Convolutional Layer
    model.add(layers.Conv1D(filters = 3 , kernel_size=3, activation='relu', name = 'C3') )

    # Flatten the CNN output to feed it with fully connected layers
    model.add(layers.Flatten())
    
    model.add(layers.Dense(48, activation='relu')) 
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(12))  # number of dense layer would be equal to number of classess
    


    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=[
              tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True, name='SparseCatCrossentropy'),
              'accuracy'])
    
    model.summary()
    
    return model